In [1]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import logging
import requests
import urllib.request
import json
import numpy as np
from tensorflow.keras.models import load_model
from gensim.models import FastText
from konlpy.tag import Okt
import boto3
import sys
import pandas as pd
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import re

if sys.version_info[0] < 3:
    from io import StringIO # Python 2.x
else:
    from io import StringIO # Python 3.x

# get your credentials from environment variables
aws_id = ''
aws_secret = ''

client = boto3.client('s3', aws_access_key_id=aws_id,
        aws_secret_access_key=aws_secret)

bucket_name = 'snucsv'

object_key = 'SNU_All.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('cp949')

# load data
train_df = pd.read_csv(StringIO(csv_string))
label_names = ["label"]
y_train = train_df[label_names].values

train_df['doc_len'] = train_df['document'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(train_df['doc_len'].mean() + train_df['doc_len'].std()).astype(int)
train_df

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


,document,link,label,doc_len
0,OECD 회원국 중 한국 터키만 상원이 없다,https://factcheck.snu.ac.kr/v2/facts/3285,전혀 사실 아님,7
1,"트럼프 ""100조 무기가 탈레반에"", 사실일까?",https://factcheck.snu.ac.kr/v2/facts/3283,대체로 사실 아님,5
2,대선후보 경선등록 후 완주하지 않으면 독자출마 가능하다,https://factcheck.snu.ac.kr/v2/facts/3281,대체로 사실 아님,7
3,사형 집행해야 사회 안전해진다,https://factcheck.snu.ac.kr/v2/facts/3279,대체로 사실 아님,4
4,사랑제일교회 야외 예배는 방역법을 준수했다,https://factcheck.snu.ac.kr/v2/facts/3278,전혀 사실 아님,5
...,...,...,...,...
4234,여름엔 긴팔 셔츠보다 반팔이 더 시원할까?,https://factcheck.snu.ac.kr/v2/facts/210,절반의 사실,6
4235,단종된 갤럭시노트7의 재고 부품을 활용한 삼성전자의 '갤럭시노트FE'는 신제품일까?,https://factcheck.snu.ac.kr/v2/facts/204,절반의 사실,8
4236,공공와이파이 20만개 설치시 연 4천800억~8천500억원의 데이터요금을 경감할 수...,https://factcheck.snu.ac.kr/v2/facts/191,대체로 사실,9
4237,참여연대 등이 '통신 기본료 1만1천원 폐지' 공약 이행을 주장하고 있다. L...,https://factcheck.snu.ac.kr/v2/facts/187,논쟁중,18


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

In [5]:
train_data = train_df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [6]:
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [8]:
class MyTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger

    def __call__(self, sent):
        pos = self.tagger.pos(sent)
        clean_words = []  # 정제된 단어 리스트
        for word in pos:
            # word[1]은 품사를 의미하며, 여기서는 조사, 문장기호, 접두사, Foreign('\n'을 빼주기 위함)인 것은 제외시킴.
            if word[1] not in ['Josa', 'Punctuation', 'Suffix', 'Foreign']:
                if len(word[0]) >= 2:  # 한 글자인 단어들도 의미가 없는 경우가 많으므로 일단 제외.
                    #if word[0] not in ['있다', '했다', '한다', '없다', '된다']:
                    clean_words.append(word[0])
        return clean_words


my_Tokenizer = MyTokenizer(Okt())

def tfidf_Vectorizer(response):
    response_list = []
    response_list.append(response)
    tfidf_Vectorizer = TfidfVectorizer(tokenizer=my_Tokenizer, min_df=1) # df 값(단어가 몇 문장들에서 등장하였는지)을 최소 'min_df' 값으로 설정.
    X = tfidf_Vectorizer.fit_transform(response_list).toarray()
#     print(X.shape)    # X(2차원 배열)의 행,열 수를 출력.
#     print(tfidf_Vectorizer.vocabulary_)   # 각 단어들이 배열 X에서 몇번째 열(인덱스 값)에 해당하는지 출력.


    #pandas를 활용하여 각 단어들의 각 문장에서의 tf-idf 값들을 모두 더하고, 내림차순으로 정렬하여 상위 n개 출력
    count = X.sum(axis=0)    # 2차원 배열 X에서 각 열을 기준으로 합을 구함. (각 단어들의 '최종' tf-idf 값으로 간주.)
    word_count = pd.DataFrame({
        '단어' : tfidf_Vectorizer.get_feature_names(),
        '빈도' : count.flat
    })
    sorted_df = word_count.sort_values('빈도', ascending=False)
#     print(sorted_df.head(10), "\n")

    word_ = list(np.array(sorted_df['단어'].tolist()))
    return(word_)

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'}

okt = Okt()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

title_list = []
for sentence in tqdm(train_data['document']):
    response = re.compile("[^A-Za-z0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]+").sub('',sentence)
    response = okt.morphs(response, stem=True) # 토큰화
    response = [word for word in response if not word in stopwords] # 불용어 제거
    response = ' '.join(response)

    word_ = tfidf_Vectorizer(response)
    word__ = ""
    if len(word_) <= 10:
        word__ = response
    else:
        for i in range(1, min(len(word_), 11)):
            word__ += word_[i]
            word__ += " "
            
    baseurl = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query='
    url = baseurl + urllib.parse.quote_plus(word__) 

    res = requests.get(url, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'html.parser')
    title_ = soup.find_all(class_ = 'news_tit')
    
    for i in range(1,30):
        try:
            summary = soup.select_one('#sp_nws{} > div > div > div.news_dsc > div > a'.format(i)).get_text()
            title_list.append(summary)
        except:
            pass

    for i in title_:
        title_list.append(i.attrs['title'])
    
len(title_list)

100%|██████████| 4239/4239 [27:09<00:00,  2.60it/s]


23957

In [9]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()
tokenized_data = []
for sentence in train_data['document']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)
    
for sentence in tqdm(title_list):
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)

100%|██████████| 23957/23957 [04:24<00:00, 90.48it/s] 


In [10]:
from gensim.models import FastText
model = FastText(sentences = tokenized_data, window = 5, min_count = 5, workers = 4, sg = 1)

model.save('similar_keyword_model_SNU_Naver')

In [12]:
loaded_model = FastText.load('similar_keyword_model_SNU_Naver')
print(loaded_model.wv.most_similar('김정은', topn=5))

[('남매', 0.8166605234146118), ('김정일', 0.7965680956840515), ('김여정', 0.7960783243179321), ('남한', 0.7897870540618896), ('탈레반', 0.7823539972305298)]
